In [3]:
import pandas as pd
import numpy as np

In [83]:

class Graph:

    def __init__(self, filename):
        
        pair_df = pd.read_csv(filename, names=["follower", "followee"])
        
        # the set of followers and followee
        self.follower_set = set(pair_df["follower"])
        self.followee_set = set(pair_df["followee"])
        
        # the set of people who are not followed by anyone
        self.not_followed_set = self.follower_set - self.followee_set
     
        # the set of vertex in the graph, and its count
        self.vertices = self.follower_set | self.followee_set
        self.num_vertices = len(self.vertices)

        # create a 2-col df [user, number of people this user follows]
        self.follow_count = pair_df.groupby("follower").count().reset_index().rename(columns={"followee": "count"})
        
        # finally, get a 3-col df [follower, followee, count]
        # where count is the total number of people that the follower follows  
        self.followers = pair_df.merge(self.follow_count, on="follower")

    def compute_ranks( self, dumping_factor = 0.15, n_iter = 10 ):

        # a constant 1 / nVertices
        uniform_weight = 1.0/self.num_vertices

        # the initial ranks (we will reassign this rdd during the computation)
        ranks = pd.DataFrame({"follower": list(self.vertices), "rank": uniform_weight})

        # the random surf term (because sometimes we just go to any random page)
        random_surf_term = dumping_factor * uniform_weight

        # the weight what gives the dangling users to the other users
        # the word 'dangling' refers to users who are followee but not follower
        # after such a user, we assume that the surfer choose the next page randomly
        dangling_user_factor = (1-dumping_factor) * uniform_weight

        # main loop: n_iter iterations
        for i in range(n_iter):
            
            # get a df [follower, rank, followee, count]
            # and reduce it to get the ranks from the edges of the graph only
            joined_ranks = ranks.merge(self.followers, how="left", on="follower")        

            # compute the sum of rank of dangling users
            dangling_user_rank_sum = joined_ranks[pd.isna(joined_ranks["count"])]["rank"].sum()
            
            # now, we have everything to update the ranks 
            # every entry (follower, followee) adds a weight (rank / count) to the new rank of followee,
            # where rank is the rank of followers and count the total number of people that follower follows
            joined_ranks.dropna(inplace=True)                          
            joined_ranks["rank"] = np.divide(joined_ranks["rank"], joined_ranks["count"])
            ranks = joined_ranks.groupby("followee")["rank"].sum().reset_index()
            ranks.columns = ["follower", "rank"]

            # add random surfer factor and dangling user factors
            dangling_user_term = dangling_user_rank_sum * dangling_user_factor
            default_term = dangling_user_term + random_surf_term
            ranks["rank"] = ranks["rank"].map(lambda rank: default_term + (1-dumping_factor) * rank)
            
           # then add the ranks for peoples who are not followed by anyone
            not_followed_df = pd.DataFrame({"follower": list(self.not_followed_set), "rank": default_term})
            
            # concatenate the result with the fixed ranks (rank of people who are not followed)
            ranks = pd.concat([ranks, not_followed_df])

        return ranks.sort_values(by=["rank", "follower"], ascending=[0, 1]).reset_index().drop("index", axis=1)

In [84]:
graph = Graph("followerFolloweeRomantic.txt")
ranks = graph.compute_ranks()
print(ranks.head(200))

                             follower      rank
0                            Composer  0.049691
1                       Baroque music  0.009389
2               Johann Sebastian Bach  0.008836
3                Ludwig van Beethoven  0.007183
4             Wolfgang Amadeus Mozart  0.006231
5                      Richard Wagner  0.005959
6                      Franz Schubert  0.005082
7                     Igor Stravinsky  0.005080
8              George Frideric Handel  0.004854
9                   Arnold Schoenberg  0.004809
10                        Franz Liszt  0.004673
11                       Joseph Haydn  0.004433
12   Giovanni Pierluigi da Palestrina  0.004327
13                    Robert Schumann  0.003935
14                 Witold Lutosławski  0.003898
15                          John Cage  0.003885
16                      Gustav Mahler  0.003834
17                  Felix Mendelssohn  0.003778
18           Classical period (music)  0.003733
19           Pyotr Ilyich Tchaikovsky  0